Importing the desired modules

In [ ]:
import numpy as np
from vtk import vtkStructuredPointsReader
from vtkmodules.util import numpy_support as VN
import vtk
from vtkmodules.util.numpy_support import vtk_to_numpy
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
from scipy import ndimage

Reading the vtk file 

In [ ]:
reader = vtkStructuredPointsReader()
reader.SetFileName("100G_lndem_3D.vtk")  #set the path to vtk file
reader.ReadAllVectorsOn()
reader.ReadAllScalarsOn()
reader.Update()

data = reader.GetOutput()
dim = data.GetDimensions()
Ox,Oy,Oz=data.GetOrigin()
Sx,Sy,Sz=data.GetSpacing()


#Variables of the contours
# Note: here the contour variables will be whatever the heading of the column of the contours in the vtk files is
u = VN.vtk_to_numpy(data.GetPointData().GetArray('lndem_3d'))
b = VN.vtk_to_numpy(data.GetPointData().GetArray('log10T'))

u = u.reshape(dim,order='F')
b = b.reshape(dim,order='F')

In [ ]:
print(dim[2])
print(Oz + Sz*(dim[2]-1)) #prints origin of z-axis, hence verifying the run

Extracting a smaller subset to work in a particular specified domain

In [ ]:
#You can choose to work on a subset rather than whole domain

#axes points, similarly for u and b
x = np.zeros(data.GetNumberOfPoints())
y = np.zeros(data.GetNumberOfPoints())
z = np.zeros(data.GetNumberOfPoints())

for i in range(data.GetNumberOfPoints()):
        x[i],y[i],z[i] = data.GetPoint(i)

#reshaping them into appripriate dimensions
x = x.reshape(dim,order='F')
y = y.reshape(dim,order='F')
z = z.reshape(dim,order='F')

x_sub=x[0:301, 110:730, 0:61]
y_sub=y[0:301, 110:730, 0:61]
z_sub=z[0:301, 110:730, 0:61]

print(np.shape(x))
print(np.shape(y))
print(np.shape(z))

Making the 2D slash cut at a specified height 

In [ ]:
#If you want to make a 2D slash at some specified point in the space you can use this slicing

x_slash=x[0:301, 150:151, 0:61]
y_slash=y[0:301, 150:151, 0:61]
z_slash=z[0:301, 150:151, 0:61]
b_slash=b[0:301, 150:151, 0:61]
u_slash=u[499:501, 200:202, 4:6]

Plotting the contours manually using the contour function of matplotlib and using the default color scheme offered by the function

In [ ]:
#grange function replaced with arange in python while bytscl is replaced 
#with np.clip and astype, np.floor is used for refoming and levels is used to clor scheming

# Assuming lndem_3D is a numpy array containing the data
lndem_3D=u
nx, nz, nt = lndem_3D.shape

# Reform and bytscl the data  
data = np.clip(np.floor(255 * (lndem_3D[0:nx-1, 400, 0:nt-1] + 3) / 3), 0, 255).astype(np.uint8)

# Set up x and time arrays
x = np.arange(nx-1)
time = np.arange(nt-1)
[X,Y]=np.meshgrid(time,x)


# Set up contour levels
levels = np.arange(0, 255, 200)

# Plot the contour
cont = plt.contour(X,Y, data, levels=levels)

# Show the plot
plt.xlabel("Time indices")
plt.ylabel("Width indices")
plt.show()

Full code to read the vtk file but using the imshow function to plot the thermal plots rather than contour plots

In [ ]:
def patch_count(slice_data, colors, id):
    # Define the threshold value for brightness
    threshold = 0.0 # This condition specifies the values at which to compute the isosurfaces

    # Apply the threshold to create a binary mask for bright spots
    binary_mask = np.where(slice_data > threshold, 1, 0)

    # Label the connected components (patches)
    labeled_mask, num_labels = ndimage.label(binary_mask)

    # Exclude the background label (0)
    num_patches = num_labels - 1

    if num_patches >0:
        return num_patches
    else:
        return 0

In [ ]:
# Get the data from the VTK file
data = reader.GetOutput()
dims = data.GetDimensions()
vtk_array = data.GetPointData().GetScalars()
np_array = vtk_to_numpy(vtk_array)
np_array = np_array.reshape(dims[2], dims[1], dims[0])
Origin=data.GetOrigin()
Spacing=data.GetSpacing()

# Define the color scheme
colors = plt.cm.hot_r

# Create an empty list or DataFrame to store the values
Spic_count = []; H_ind = []; H_Mm = []

# Perform the calculation in a while loop and append the results to the list or DataFrame
gap=3; start=180; i=0; check=0
while check == 0:
    id=start + gap*i
    slice_data=np_array[:, id, :]
    spic=patch_count(slice_data, colors, id)

    #formula to find the maximum height
    H_max= Oy + Sy*(id-gap) 

    #Appending the data accrodingly
    Spic_count.append(spic)
    H_ind.append(id)
    H_Mm.append(Oy + (id)*Sy)
    if spic == 0 or H_max >= 28:
        check=1
    i+=1

# Create a dictionary with column names and their values
data_to_csv = {'Spicules': Spic_count,
        'Height[i]': H_ind,
        'Height (Mm)': H_Mm}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data_to_csv)

# Specify the file path and name for your Excel sheet
file_path = "100G_lndem_3D_48km_data.csv"

# Write the DataFrame to the Excel file using pandas
df.to_csv(file_path, index=False)

In [ ]:
#Printing the height at whih the last spicules was counted    
if Spic_count[-1]>0:
    print(round(H_Mm[-1],3))
elif Spic_count[-2]>0:
    print(round(H_Mm[-2],3))

In [ ]:
slice_data=np_array[:, 400, :]

# Plot the slice data with the color scheme
print("Without specifying the color scheme")
plt.imshow(slice_data, cmap=colors)
plt.colorbar()
plt.show()

# Define the custom grayscale color scheme
colors_list = [(i/255.0, i/255.0, i/255.0) for i in range(256)]
values_list = [i for i in range(256)]
colors = ListedColormap(colors_list)

# Define the custom color scheme
'''colors_list = [(0.0, 0.0, 0.0), (1.0, 0.0, 0.0), (1.0, 1.0, 0.0),(1.0, 1.0, 1.0)]
values_list = [0, 10, 20, 30]
colors = ListedColormap(colors_list)'''

# Plot the slice data with the customized color scheme
print("After specifying the color scheme")
#extent is for resizing the axes and aspect is the ratio
plt.imshow(slice_data, cmap=colors, vmin=0, vmax=2, extent=[Origin[0], Origin[0] + Spacing[0]*(dims[0]-1), Origin[2], Origin[2] + Spacing[2]*(dims[2]-1)],aspect=1) 
#plt.colorbar()
plt.xlabel("$x$ (width in Mm)",fontsize=12)
plt.ylabel("$t$ (time in min)",fontsize=12)
plt.colorbar(orientation='horizontal'). set_label("$log(I/I_0)$")
plt.show()

print(Origin)